In [1]:
# import modules
import psycopg2 # pip install psycopg2
import json
import csv

In [3]:
# clean raw_seasons.json data & convert to csv

with open('./data/raw_seasons.json', encoding="utf-8") as file:
    seasons = json.load(file)

# csv headers
seasons_data = [["seasonID","seasonYear","startDate","endDate","leagueName","leagueType","leagueCountry"]]
    
for season in seasons:
    seasons_data.append([
        season["seasonID"],
        season["year"], 
        season["start"], 
        season["end"],
        season["name"],
        season["type"],
        season["country"]
    ])
    
with open('./data/cleaned_seasons.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(seasons_data)

In [13]:
# insert cleaned_seasons.csv data into db

# connect to db
connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# before move data to public, because of access rights
sql = '''COPY Season(seasonID,seasonYear,startDate,endDate,leagueName,leagueType,leagueCountry)
FROM 'C:/Users/Public/cleaned_seasons.csv'
DELIMITER ','
CSV HEADER;'''

cursor.execute(sql)

# close connection
cursor.close()
connection.close()

In [6]:
# helper to check if birthdate of player is a valid birthdate

import calendar
from datetime import datetime

def is_valid_birthdate(birthdate):
    # Split the birthdate into its components
    year, month, day = map(int, birthdate.split('-'))

    # Check that the month is between 1 and 12, and the day is between 1 and the number of days in the month
    if not (1 <= month <= 12) or not (1 <= day <= calendar.monthrange(year, month)[1]):
        return False

    # Check that the year is not in the future
    current_year = datetime.now().year
    if year > current_year:
        return False

    return True

def is_valid_primary_keys(playerId, firstname, lastname, birthdate):
    values = [playerId, firstname, lastname, birthdate]
    # if all values are not None, return True
    return all(value is not None for value in values)

def is_duplicate(firstname, lastname, birthdate, dictionary):
    key = firstname + lastname + birthdate
    if key in dictionary:
        return True # duplicate found
    return False

def manipulate_wh(weightInput, heightInput):
    weight, height = weightInput, heightInput
     # give weight and height zero value if null
    if weight is not None:
        weight_str = weight.strip()
        if weight_str != "":
            weight = int(weight.replace("kg", ""))
        else:
            weight = 0
    else:
        weight = 0
    if height is not None:
        height_str = height.strip()
        if height_str != "":
            height = int(height.replace("cm", ""))
        else:
            height = 0
    else:
        height = 0
        
    return weight, height

def is_valid_primary_keys_stats(playerId, seasonId, teamId):
    values = [playerId, seasonId, teamId]
    # if all values are not None, return True
    return all(value is not None for value in values)

def is_duplicate_stats(playerId, seasonId, teamId, dictionary):
    key = str(playerId) + str(seasonId) + str(teamId)
    if key in dictionary:
        return True # duplicate found
    return False

def check_string(arr, toCheck = "\n    "):
    values = arr
    for i in range(len(values)):
        if values[i] == toCheck:
            values[i] = None
    return values

In [12]:
# clean raw_players.json data & convert to csv

# raw players (with statistics) file
with open('./data/raw_players.json', encoding="utf8") as file:
    players = json.load(file)

# csv headers
players_data = [["playerID","firstname","lastname","birthDate","heightCm","weightKg","nationality"]]
stats_data = [[
    "playerID","seasonID","teamID","teamName","gamesAppearances", 
    "gamesLineups","gamesMinutes","gamesPosition","gamesCaptain", 
    "gamesRating","goalsTotal","goalsConceded","goalsAssists",
    "goalsSaved","tacklesTotal","tacklesBlocks","tacklesInterceptions", 
    "foulsDrawn","foulsCommited","passesKey","passesTotal","passesAccuracy", 
    "duelsWon","duelsTotal","cardRed","cardYellow","cardYellowred","shotsTotal",
    "shotsOn","dribblesAttempts", "dribblesSuccess","dribblesPast","penaltyWon", 
    "penaltyCommited","penaltyScored","penaltyMissed","penaltySaved",
    "substituesIn", "substitutesOut","substitutesBench"]]
playsIn_data = [["playerID", "seasonID"]]

# hash table
players_dict = {}

skipped_players = 0
skipped_stats = 0

for player in players:
    playerId, seasonId, firstname, lastname, birthdate, statistics = player["player"]["id"], player["seasonID"], player["player"]["firstname"], player["player"]["lastname"], player["player"]["birth"]["date"], player["statistics"]
    
    # add players only if they have an id, firstname, lastname and a birthday
    if not is_valid_primary_keys(playerId, firstname, lastname, birthdate):
        skipped_players += 1
        continue
    
    # don't add add players without a valid birthday
    if not is_valid_birthdate(birthdate):
        skipped_players += 1
        continue
        
    # append player only if no duplcate found, append stats anyway
    if not is_duplicate(firstname, lastname, birthdate, players_dict):
        # manipulate weight and height
        weight, height = manipulate_wh(player["player"]["weight"], player["player"]["height"])

        # append to final players data
        players_data.append([
            playerId,
            firstname, 
            lastname, 
            birthdate,
            height,
            weight,
            player["player"]["nationality"]
        ])

        # append key to players_dict to check in next iteration if duplicate
        player_key = firstname + lastname + birthdate
        players_dict[player_key] = True
        players_dict[playerId] = True # Foreign Key Constraint

        # append to final playsIn data
        playsIn_data.append([
            playerId,
            seasonId, 
        ])
    else:
        if playerId in players_dict:
            # append in playsIn otherwise playsIn only with one playerid record
            playsIn_data.append([
                playerId,
                seasonId, 
            ])
        
    # if stats are not empty or not valid continue
    temp_stats_dict = {}
    for stats in statistics:
        if not is_valid_primary_keys_stats(playerId, seasonId, stats["team"]["id"]):
            skipped_stats += 1
            continue
        if is_duplicate_stats(playerId, seasonId, stats["team"]["id"], temp_stats_dict):
            skipped_stats += 1
            continue
            
        # if player is not added to players data / foreign key constraint skip stats
        player_key = firstname + lastname + birthdate
        if not (player_key in players_dict):
            skipped_stats += 1
            continue
            
        if not (playerId in players_dict):
            skipped_stats += 1
            continue
            
        # append key to temp stats to check in next iteration if duplicate
        stats_key = str(playerId) + str(seasonId) + str(stats["team"]["id"])
        temp_stats_dict[stats_key] = True
        
        data = [
            playerId,
            seasonId, 
            stats["team"]["id"], 
            stats["team"]["name"],
            stats["games"]["appearences"],
            stats["games"]["lineups"],
            stats["games"]["minutes"],
            stats["games"]["position"], 
            stats["games"]["captain"], 
            stats["games"]["rating"],
            stats["goals"]["total"], 
            stats["goals"]["conceded"], 
            stats["goals"]["assists"], 
            stats["goals"]["saves"], 
            stats["tackles"]["total"], 
            stats["tackles"]["blocks"],
            stats["tackles"]["interceptions"], 
            stats["fouls"]["drawn"], 
            stats["fouls"]["committed"], 
            stats["passes"]["key"], 
            stats["passes"]["total"],
            stats["passes"]["accuracy"], 
            stats["duels"]["won"], 
            stats["duels"]["total"], 
            stats["cards"]["red"], 
            stats["cards"]["yellow"], 
            stats["cards"]["yellowred"],
            stats["shots"]["total"], 
            stats["shots"]["on"], 
            stats["dribbles"]["attempts"], 
            stats["dribbles"]["success"], 
            stats["dribbles"]["past"],
            stats["penalty"]["won"], 
            stats["penalty"]["commited"], 
            stats["penalty"]["scored"], 
            stats["penalty"]["missed"], 
            stats["penalty"]["saved"],
            stats["substitutes"]["in"], 
            stats["substitutes"]["out"], 
            stats["substitutes"]["bench"],
        ]
        
        # check if values include "\n    "
        stats_values = check_string(data)
        
        # append to final stats data
        stats_data.append(stats_values)
        
print("Skipped players due to not tidy data: ", skipped_players)
print("Skipped stats due to not tidy data: ", skipped_stats)

with open('./data/cleaned_players.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(players_data)
    
with open('./data/cleaned_statistics.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(stats_data)
    
with open('./data/cleaned_playsIn.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(playsIn_data)

Skipped players due to not tidy data:  78174
Skipped stats due to not tidy data:  9538


In [14]:
# insert cleaned_players.csv data into db

# connect to db
connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# move data to public, because of access rights (ON CONFLICT clause is only available in PostgreSQL 9.5 and later)
sql = '''COPY Player(playerID,firstname,lastname,birthDate,heightCm,weightKg,nationality)
FROM 'C:/Users/Public/cleaned_players.csv'
DELIMITER ','
ENCODING 'UTF8'
CSV HEADER;'''

try:
    cursor.execute(sql)
except Exception as e:
    print(e)

# close connection
cursor.close()
connection.close()

In [15]:
# insert cleaned_playsIn.csv data into db

# connect to db
connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# move data to public, because of access rights (ON CONFLICT clause is only available in PostgreSQL 9.5 and later)
sql = '''COPY playsIn(playerID, seasonID)
FROM 'C:/Users/Public/cleaned_playsIn.csv'
DELIMITER ','
ENCODING 'UTF8'
CSV HEADER;'''

try:
    cursor.execute(sql)
except Exception as e:
    print(e)

# close connection
cursor.close()
connection.close()

In [16]:
# insert cleaned_statistics.csv data into db

# connect to db
connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# move data to public, because of access rights (ON CONFLICT clause is only available in PostgreSQL 9.5 and later)
sql = '''COPY Statistics(playerID,seasonID,teamID,teamName,gamesAppearances,gamesLineups,gamesMinutes,gamesPosition,gamesCaptain,gamesRating,goalsTotal,goalsConceded,goalsAssists,
goalsSaved,tacklesTotal,tacklesBlocks,tacklesInterceptions,foulsDrawn,foulsCommited,passesKey,passesTotal,passesAccuracy,duelsWon,duelsTotal,cardRed,cardYellow,cardYellowred,shotsTotal,
shotsOn,dribblesAttempts,dribblesSuccess,dribblesPast,penaltyWon,penaltyCommited,penaltyScored,penaltyMissed,penaltySaved,substituesIn,substitutesOut,substitutesBench)
FROM 'C:/Users/Public/cleaned_statistics.csv'
DELIMITER ','
ENCODING 'UTF8'
CSV HEADER;'''

try:
    cursor.execute(sql)
except Exception as e:
    print(e)

# close connection
cursor.close()
connection.close()

In [17]:
# further helper functions
def transform_birthdate(bday):
    if bday == None:
        return "3000-01-01" # return invalid birthdate
    return bday[:10]

In [18]:
# 1. Matching: Match cards with primary keys firstname, lastname and birthdate

with open('./data/raw_sorare_all_cards.json', encoding="utf8") as file:
    cards = json.load(file)
    
# csv headers
cards_data = [["assetID","rarity","seasonYear","birthDate","bestFoot","firstName","lastName", "shirtNumber"]]
not_appended_cards_data = []

# hash tables to check for duplicates
cards_dict = {}

# helper to count
skipped_cards = 0


print("Length of cards json data: ", len(cards))
for card in cards:
    assetId, firstname, lastname, birthdate = card["assetId"], card["player"]["firstName"], card["player"]["lastName"], card["player"]["birthDate"]
    
    # add players only if they have an id, firstname, lastname and a birthday
    if not is_valid_primary_keys(assetId, firstname, lastname, birthdate):
        skipped_cards += 1
        continue
    
    # transform & check birthdate
    birthdate = transform_birthdate(birthdate)
    if not is_valid_birthdate(birthdate):
        skipped_cards += 1
        continue
        
    # don't add duplicates
    if assetId in cards_dict:
        skipped_cards += 1
        continue
        
    # foreign key constraint
    player_key = firstname + lastname + birthdate #from cards
    if player_key in players_dict: # match
        cards_data.append([
            card["assetId"],
            card["rarity"],
            card["season"]["startYear"],
            birthdate,
            card["player"]["bestFoot"],
            firstname,
            lastname,
            card["player"]["shirtNumber"]
        ])
        cards_dict[card["assetId"]] = True # to check for duplicates (here & later on)
    else: # no match
        not_appended_cards_data.append([
            card["assetId"],
            card["rarity"],
            card["season"]["startYear"],
            birthdate,
            card["player"]["bestFoot"],
            firstname,
            lastname,
            card["player"]["shirtNumber"],
            card["player"]["weight"],
            card["player"]["height"]
        ])
        

print("Skipped cards due to not tidy data: ", skipped_cards)
print("Total cards: ", len(cards_data) + len(not_appended_cards_data))
print("Matched cards: ", len(cards_data))
print("Not appended cards due to not matched data: ", len(not_appended_cards_data))    

with open('./data/cleaned_cards_1.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(cards_data)
    
with open('./data/cleaned_not_appended_cards_1.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(not_appended_cards_data)

Length of cards json data:  1024204
Skipped cards due to not tidy data:  1
Total cards:  1024204
Matched cards:  502275
Not appended cards due to not matched data:  521929


In [19]:
# insert 1. Match cleaned_cards_1.csv data into db

# connect to db
connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# move data to public, because of access rights (ON CONFLICT clause is only available in PostgreSQL 9.5 and later)
sql = '''COPY Card(assetID,rarity,seasonYear,birthDate,bestFoot,firstName,lastName,shirtNumber)
FROM 'C:/Users/Public/cleaned_cards_1.csv'
DELIMITER ','
ENCODING 'UTF8'
CSV HEADER;'''

try:
    cursor.execute(sql)
except Exception as e:
    print(e)

# close connection
cursor.close()
connection.close()

In [20]:
# create hash table with not appended cards
not_appended_cards_dict = {}
for card in not_appended_cards_data:
    assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
    player_key = firstname + lastname + birthdate
    if player_key in not_appended_cards_dict:
        not_appended_cards_dict[player_key].append((assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height))
    else:
        not_appended_cards_dict[player_key] = [(assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height)]
        
print("No of players that are not matched as cards: ", len(not_appended_cards_dict))
print("with a total of no of rows that are not matched: ", len(not_appended_cards_data))

No of players that are not matched as cards:  5626
with a total of no of rows that are not matched:  521929


In [21]:
# helper

def is_valid_elements(birthdate, weight, height):
    values = [birthdate, weight, height]
    # if all values are not None, return True
    return all(value is not None for value in values)

In [22]:
# 2. Matching: Match cards with birthdate, weight and height
# Assumption/Idea: A professionel player has a sorare card with correct weight, height and birthdate. This information should also be the same in our player table

connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

cards_data_v2 = [["assetID","rarity","seasonYear","birthDate","bestFoot","firstName","lastName", "shirtNumber"]]
not_appended_cards_data_v2 = [] # cards that still are not going to be appended
potential_appendable_cards_data = []

new_cards_dict = dict(cards_dict) # to check for duplicates in play

# helper to count
not_valid_elements = 0
duplicate_found_rows = 0
r0 = 0
r1 = 0
r2 = 0


for key, value in not_appended_cards_dict.items():
    assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = value[0][0], value[0][1], value[0][2], value[0][3], value[0][4], value[0][5], value[0][6], value[0][7], value[0][8], value[0][9]

    # if one of the values are null, don't query it
    if not is_valid_elements(birthdate, weight, height):
        for card in value:
            assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
            not_appended_cards_data_v2.append([
                assetId,
                rarity,
                seasonYear,
                birthdate,
                bestFoot,
                firstname,
                lastname,
                shirtNumber,
                weight,
                height
            ])
        not_valid_elements += 1
        continue
    
    # make sql statement (assumpiton: professional player, weight and height match)
    query = "SELECT * FROM Player WHERE birthdate = %s AND weightkg = %s AND heightcm = %s"
    cursor.execute(query, (birthdate, weight, height))
    results = cursor.fetchall()
    
    # if we couldn't find player in our players table with same birthdate, height & weight
    if len(results) == 0:
        for card in value:
            assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
            not_appended_cards_data_v2.append([
                assetId,
                rarity,
                seasonYear,
                birthdate,
                bestFoot,
                firstname,
                lastname,
                shirtNumber,
                weight,
                height
            ])
        r0 += 1
        continue
    
    # perfect match
    if len(results) == 1:
        temp_r1 = 0
        for card in value:
            assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
            
            # don't add duplicates
            if assetId in new_cards_dict:
                duplicate_found_rows += 1
            else:
                cards_data_v2.append([
                    assetId,
                    rarity,
                    seasonYear,
                    birthdate, # birthdate must be correct, coz of query - foreign key constrain
                    bestFoot,
                    results[0][1], # use firstname from players table (source of truth) - foreign key constrain
                    results[0][2], # use lastname from players table (source of truth) - foreign key constrain
                    shirtNumber
                ])

                new_cards_dict[assetId] = True
                temp_r1 += 1
        if temp_r1 > 0: r1 += 1
    else:
        for card in value:
            assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
            potential_appendable_cards_data.append([
                assetId,
                rarity,
                seasonYear,
                birthdate,
                bestFoot,
                firstname,
                lastname,
                shirtNumber,
                weight,
                height
            ])
        r2 += 1
        continue
        
# close connection
cursor.close()
connection.close()
        
print("Players without valid elements: ", not_valid_elements)
print("Players without results in players table: ", r0)
print("These player can potentially be matched: ", not_valid_elements + r0, " with a total no of rows: ", len(not_appended_cards_data_v2))
print("******************")
print("Players (rows) that are matched, but already in cards table (no action required): ", duplicate_found_rows)
print("******************")
print("Players that couldn't be matched, because there at least two players with same birthdate, weight & height: ", r2)
print("with total rows that could still be matched with other methods: ", len(potential_appendable_cards_data))
print("******************")
print("Players that are now matched: ", r1)
print("with new added total rows of: ", len(cards_data_v2))
print("******************")
print("At this point we matched from a total of ", len(cards_data) + len(cards_data_v2), " rows.")


with open('./data/cleaned_cards_2.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(cards_data_v2)
    
with open('./data/cleaned_not_appended_cards_2.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(not_appended_cards_data_v2)
    
with open('./data/cleaned_potentially_appendable_cards_2.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(potential_appendable_cards_data)

Players without valid elements:  1831
Players without results in players table:  1462
These player can potentially be matched:  3293  with a total no of rows:  299998
******************
Players (rows) that are matched, but already in cards table (no action required):  3
******************
Players that couldn't be matched, because there at least two players with same birthdate, weight & height:  95
with total rows that could still be matched with other methods:  9026
******************
Players that are now matched:  2238
with new added total rows of:  212903
******************
At this point we matched from a total of  715178  rows.


In [23]:
# insert 2. Match cleaned_cards_2.csv data into db

# connect to db
connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# move data to public, because of access rights (ON CONFLICT clause is only available in PostgreSQL 9.5 and later)
sql = '''COPY Card(assetID,rarity,seasonYear,birthDate,bestFoot,firstName,lastName,shirtNumber)
FROM 'C:/Users/Public/cleaned_cards_2.csv'
DELIMITER ','
ENCODING 'UTF8'
CSV HEADER;'''

try:
    cursor.execute(sql)
except Exception as e:
    print(e)

# close connection
cursor.close()
connection.close()

In [24]:
# create hash table for third matching
potential_appendable_cards_dict = {}
for card in potential_appendable_cards_data:
    assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
    player_key = firstname + lastname + birthdate
    if player_key in potential_appendable_cards_dict:
        potential_appendable_cards_dict[player_key].append((assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height))
    else:
        potential_appendable_cards_dict[player_key] = [(assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height)]
        
print("No of players that are still not matched as cards: ", len(potential_appendable_cards_dict))
print("with a total of no of rows that are not matched: ", len(potential_appendable_cards_data))

No of players that are still not matched as cards:  95
with a total of no of rows that are not matched:  9026


In [25]:
# 3. Matching (95 Player, 9026 rows): Match cards with name
# results of queries are always bigger than 1

from Levenshtein import distance

cards_data_v3 = [["assetID","rarity","seasonYear","birthDate","bestFoot","firstName","lastName", "shirtNumber"]]

new_cards_dict_v3 = dict(new_cards_dict) # to check for duplicates in play

connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# helper
matches = 0
added_matches = 0
duplicate_found_rows = 0

for key, value in potential_appendable_cards_dict.items():
    birthdate, firstname, lastname, weight, height = value[0][3], value[0][5], value[0][6], value[0][8], value[0][9]
    cardFirstname, cardLastname = firstname.split(' ')[0], lastname.split(' ')[0]
    cardName = cardFirstname + cardLastname
    
    # make sql statement (assumpiton: professional player, weight and height match)
    query = "SELECT * FROM Player WHERE birthdate = %s AND weightkg = %s AND heightcm = %s"
    cursor.execute(query, (birthdate, weight, height))
    results = cursor.fetchall()
    
    for result in results:
        # players table
        playerFirstname, playerLastname = result[1].split(' ')[0], result[2].split(' ')[0]
        playerName = playerFirstname + playerLastname
        
        # find Levenshtein distance between cards name and players name in Player table
        lev_distance = distance(cardName, playerName)
        
        if lev_distance < 5:
            matches += 1
            temp_r1 = 0
            for card in value:
                assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]

                # don't add duplicates
                if assetId in new_cards_dict_v3:
                    duplicate_found_rows += 1
                else:
                    cards_data_v3.append([
                        assetId,
                        rarity,
                        seasonYear,
                        birthdate, # birthdate must be correct, coz of query - foreign key constrain
                        bestFoot,
                        result[1], # use firstname from players table (source of truth) - foreign key constrain
                        result[2], # use lastname from players table (source of truth) - foreign key constrain
                        shirtNumber
                    ])

                    new_cards_dict_v3[assetId] = True
                    temp_r1 += 1
            if temp_r1 > 0: added_matches += 1
    
    
# close connection
cursor.close()
connection.close()

with open('./data/cleaned_cards_3.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(cards_data_v3)
    
    
print("We found in total ", matches, " matches.")
print("We added in total ", added_matches, " matches with total rows of: ", len(cards_data_v3))
print("Duplicate rows: ", duplicate_found_rows)

We found in total  71  matches.
We added in total  70  matches with total rows of:  6612
Duplicate rows:  56


In [26]:
# insert 3. Match cleaned_cards_3.csv data into db

# connect to db
connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# move data to public, because of access rights (ON CONFLICT clause is only available in PostgreSQL 9.5 and later)
sql = '''COPY Card(assetID,rarity,seasonYear,birthDate,bestFoot,firstName,lastName,shirtNumber)
FROM 'C:/Users/Public/cleaned_cards_3.csv'
DELIMITER ','
ENCODING 'UTF8'
CSV HEADER;'''

try:
    cursor.execute(sql)
except Exception as e:
    print(e)

# close connection
cursor.close()
connection.close()

In [27]:
# create hash table with not appended cards v2
not_appended_cards_dict_v2 = {}
for card in not_appended_cards_data_v2:
    assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
    player_key = firstname + lastname + birthdate
    if player_key in not_appended_cards_dict_v2:
        not_appended_cards_dict_v2[player_key].append((assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height))
    else:
        not_appended_cards_dict_v2[player_key] = [(assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height)]
        
print("No of players that are not matched as cards: ", len(not_appended_cards_dict_v2))
print("with a total of no of rows that are not matched: ", len(not_appended_cards_data_v2))

No of players that are not matched as cards:  3293
with a total of no of rows that are not matched:  299998


In [28]:
# 4. Matching (3293 Player, 299998 rows): Match cards with name

cards_data_v4 = [["assetID","rarity","seasonYear","birthDate","bestFoot","firstName","lastName", "shirtNumber"]]
not_appended_cards_data_v3 = []
new_cards_dict_v4 = dict(new_cards_dict_v3) # to check for duplicates in play

connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# helper
matches_v2 = 0
added_matches_v2 = 0
duplicate_found_rows_v2 = 0
no_results = 0

for key, value in not_appended_cards_dict_v2.items():
    birthdate, firstname, lastname = value[0][3], value[0][5], value[0][6]
    cardFirstname, cardLastname = firstname.split(' ')[0], lastname.split(' ')[0]
    cardName = cardFirstname + cardLastname
    
    # make sql statement (assumpiton: professional player, weight and height match)
    query = "SELECT * FROM Player WHERE birthdate = %s"
    cursor.execute(query, (birthdate,))
    results = cursor.fetchall()
    
    if len(results) == 0:
        for card in value:
            assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
            not_appended_cards_data_v3.append([
                assetId,
                rarity,
                seasonYear,
                birthdate,
                bestFoot,
                firstname,
                lastname,
                shirtNumber,
                weight,
                height
            ])
        no_results += 1
        continue
        
    temp_r2 = 0
        
    for result in results:
        # players table
        playerFirstname, playerLastname = result[1].split(' ')[0], result[2].split(' ')[0]
        playerName = playerFirstname + playerLastname
        
        # find Levenshtein distance between cards name and players name in Player table
        lev_distance = distance(cardName, playerName)
        
        # very close match (weight/height problem solved)
        if lev_distance < 3:
            matches_v2 += 1
            temp_r1 = 0
            for card in value:
                assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]

                # don't add duplicates
                if assetId in new_cards_dict_v4:
                    duplicate_found_rows_v2 += 1
                else:
                    cards_data_v4.append([
                        assetId,
                        rarity,
                        seasonYear,
                        birthdate, # birthdate must be correct, coz of query - foreign key constrain
                        bestFoot,
                        result[1], # use firstname from players table (source of truth) - foreign key constrain
                        result[2], # use lastname from players table (source of truth) - foreign key constrain
                        shirtNumber
                    ])

                    new_cards_dict_v4[assetId] = True
                    temp_r1 += 1
                    temp_r2 += 1
            if temp_r1 > 0: added_matches_v2 += 1
    
    # card wasn't matched or duplicate
    if temp_r2 == 0: 
        for card in value:
            assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
            not_appended_cards_data_v3.append([
                assetId,
                rarity,
                seasonYear,
                birthdate,
                bestFoot,
                firstname,
                lastname,
                shirtNumber,
                weight,
                height
            ])
                
# close connection
cursor.close()
connection.close()

with open('./data/cleaned_cards_4.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(cards_data_v4)
    
with open('./data/cleaned_not_appended_cards_3.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(not_appended_cards_data_v3)
    
    
print("We found in total ", matches_v2, " matches.")
print("We added in total ", added_matches_v2, " matches with total rows of: ", len(cards_data_v4))
print("Duplicate rows: ", duplicate_found_rows_v2)
print("No results (no birthdate match): ", no_results)
print("Still not matched data (rows) ", len(not_appended_cards_data_v3))

We found in total  1692  matches.
We added in total  1692  matches with total rows of:  156761
Duplicate rows:  0
No results (no birthdate match):  22
Still not matched data (rows)  143238


In [29]:
# insert 4. Match cleaned_cards_4.csv data into db

# connect to db
connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# move data to public, because of access rights (ON CONFLICT clause is only available in PostgreSQL 9.5 and later)
sql = '''COPY Card(assetID,rarity,seasonYear,birthDate,bestFoot,firstName,lastName,shirtNumber)
FROM 'C:/Users/Public/cleaned_cards_4.csv'
DELIMITER ','
ENCODING 'UTF8'
CSV HEADER;'''

try:
    cursor.execute(sql)
except Exception as e:
    print(e)

# close connection
cursor.close()
connection.close()

In [30]:
# create hash table with not appended cards v3
not_appended_cards_dict_v3 = {}
for card in not_appended_cards_data_v3:
    assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
    player_key = firstname + lastname + birthdate
    if player_key in not_appended_cards_dict_v3:
        not_appended_cards_dict_v3[player_key].append((assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height))
    else:
        not_appended_cards_dict_v3[player_key] = [(assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height)]
        
print("No of players that are not matched as cards: ", len(not_appended_cards_dict_v3))
print("with a total of no of rows that are not matched: ", len(not_appended_cards_data_v3))

No of players that are not matched as cards:  1601
with a total of no of rows that are not matched:  143238


In [31]:
# 5. Matching (1601 Player, 143238 rows): Match cards with firstname or lastname exact match

cards_data_v5 = [["assetID","rarity","seasonYear","birthDate","bestFoot","firstName","lastName", "shirtNumber"]]
not_appended_cards_data_v4 = []
new_cards_dict_v5 = dict(new_cards_dict_v4) # to check for duplicates in play

connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# helper
matches_v2 = 0
added_matches_v2 = 0
duplicate_found_rows_v2 = 0
no_results = 0

for key, value in not_appended_cards_dict_v3.items():
    birthdate, firstname, lastname = value[0][3], value[0][5], value[0][6]
    cardFirstname, cardLastname = firstname.split(' ')[0], lastname.split(' ')[0]
    
    # make sql statement (assumpiton: professional player, weight and height match)
    query = "SELECT * FROM Player WHERE birthdate = %s"
    cursor.execute(query, (birthdate,))
    results = cursor.fetchall()
    
    if len(results) == 0:
        for card in value:
            assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
            not_appended_cards_data_v4.append([
                assetId,
                rarity,
                seasonYear,
                birthdate,
                bestFoot,
                firstname,
                lastname,
                shirtNumber,
                weight,
                height
            ])
        no_results += 1
        continue
        
    temp_r2 = 0
        
    for result in results:
        # players table
        playerFirstname, playerLastname = result[1].split(' ')[0], result[2].split(' ')[0]
        
        # find Levenshtein distance between firstname and lastname seperated
        lev_distance_firstname = distance(cardFirstname, playerFirstname)
        lev_distance_lastname = distance(cardLastname, playerLastname)

        
        # exact match either firstname or lastname
        if lev_distance_firstname == 0 or lev_distance_lastname == 0:
            matches_v2 += 1
            temp_r1 = 0
            for card in value:
                assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]

                # don't add duplicates
                if assetId in new_cards_dict_v5:
                    duplicate_found_rows_v2 += 1
                else:
                    cards_data_v5.append([
                        assetId,
                        rarity,
                        seasonYear,
                        birthdate, # birthdate must be correct, coz of query - foreign key constrain
                        bestFoot,
                        result[1], # use firstname from players table (source of truth) - foreign key constrain
                        result[2], # use lastname from players table (source of truth) - foreign key constrain
                        shirtNumber
                    ])

                    new_cards_dict_v5[assetId] = True
                    temp_r1 += 1
                    temp_r2 += 1
            if temp_r1 > 0: added_matches_v2 += 1
    
    # card wasn't matched or duplicate
    if temp_r2 == 0: 
        for card in value:
            assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
            not_appended_cards_data_v4.append([
                assetId,
                rarity,
                seasonYear,
                birthdate,
                bestFoot,
                firstname,
                lastname,
                shirtNumber,
                weight,
                height
            ])
                
# close connection
cursor.close()
connection.close()

with open('./data/cleaned_cards_5.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(cards_data_v5)
    
with open('./data/cleaned_not_appended_cards_4.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(not_appended_cards_data_v4)
    
    
print("We found in total ", matches_v2, " matches.")
print("We added in total ", added_matches_v2, " matches with total rows of: ", len(cards_data_v5))
print("Duplicate rows: ", duplicate_found_rows_v2)
print("No results (no birthdate match): ", no_results)
print("Still not matched data (rows) ", len(not_appended_cards_data_v4))

We found in total  892  matches.
We added in total  861  matches with total rows of:  80615
Duplicate rows:  2427
No results (no birthdate match):  22
Still not matched data (rows)  62624


In [32]:
# insert 5. Match cleaned_cards_5.csv data into db

# connect to db
connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# move data to public, because of access rights (ON CONFLICT clause is only available in PostgreSQL 9.5 and later)
sql = '''COPY Card(assetID,rarity,seasonYear,birthDate,bestFoot,firstName,lastName,shirtNumber)
FROM 'C:/Users/Public/cleaned_cards_5.csv'
DELIMITER ','
ENCODING 'UTF8'
CSV HEADER;'''

try:
    cursor.execute(sql)
except Exception as e:
    print(e)

# close connection
cursor.close()
connection.close()

In [33]:
# create hash table with not appended cards v4
not_appended_cards_dict_v4 = {}
for card in not_appended_cards_data_v4:
    assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height = card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], card[8], card[9]
    player_key = firstname + lastname + birthdate
    if player_key in not_appended_cards_dict_v4:
        not_appended_cards_dict_v4[player_key].append((assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height))
    else:
        not_appended_cards_dict_v4[player_key] = [(assetId, rarity, seasonYear, birthdate, bestFoot, firstname, lastname, shirtNumber, weight, height)]
        
print("No of players that are not matched as cards: ", len(not_appended_cards_dict_v4))
print("with a total of no of rows that are not matched: ", len(not_appended_cards_data_v4))

No of players that are not matched as cards:  740
with a total of no of rows that are not matched:  62624


In [34]:
# clean raw_sorare_all_nfts.json data & convert to csv
with open('./data/raw_sorare_all_nfts.json', encoding="utf-8") as file:
    nfts = json.load(file)

In [35]:
nfts_data = [["nftID","assetID","slug","currentOwnerAddress"]]
ownershipHistory_data = [["address","transferDate","priceEuro","transferType", "blockchain", "nftID"]]
new_cards_dict_v6 = dict(new_cards_dict_v5)
oHistory_dict = {}

# helper
not_inserted_nfts = 0

for nft in nfts:
    
    # check if there is an owner address, don't insert
    if nft is not None and "owner" in nft and nft["owner"] is not None and "address" in nft["owner"] and nft["owner"]["address"] is not None:
        currentOwnerAddress = nft["owner"]["address"]
    else:
        not_inserted_nfts += 1
        continue

    nftId, assetId, slug, ownershipHistory = nft["id"], nft["assetId"], nft["slug"], nft["ownershipHistory"]
    
    # foreign key constraint
    if assetId in new_cards_dict_v6:
        nfts_data.append([nftId, assetId, slug, currentOwnerAddress])
        
        # insert ownershiphistory
        if len(ownershipHistory) > 0:
            for owHistory in ownershipHistory:
                address, transferDate, priceEuro, transferType, blockchain = owHistory["address"], owHistory["from"], owHistory["priceFiat"]["eur"], owHistory["transferType"], owHistory["blockchain"]
                
                # don't add 0 trx & primary key duplicate check
                key = address + transferDate
                if priceEuro > 0 and (key not in oHistory_dict):
                    ownershipHistory_data.append([address, transferDate, priceEuro, transferType, blockchain, nftId])
                    oHistory_dict[key] = True
    else:
        not_inserted_nfts += 1
        
print("All nft rows: ", len(nfts))
print("Inserted nfts (rows): ", len(nfts_data))
print("Not inserted nfts (rows): ", not_inserted_nfts)
print("Inserted owHistory rows: ", len(ownershipHistory_data))

with open('./data/cleaned_nfts.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(nfts_data)
    
with open('./data/cleaned_owHistory.csv', 'w', newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(ownershipHistory_data)
    

All nft rows:  1380900
Inserted nfts (rows):  898150
Not inserted nfts (rows):  482751
Inserted owHistory rows:  1850694


In [36]:
# insert cleaned_nfts.csv data into db

# connect to db
connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# move data to public, because of access rights (ON CONFLICT clause is only available in PostgreSQL 9.5 and later)
sql = '''COPY NFT(nftID,assetID,slug,currentOwnerAddress)
FROM 'C:/Users/Public/cleaned_nfts.csv'
DELIMITER ','
ENCODING 'UTF8'
CSV HEADER;'''

try:
    cursor.execute(sql)
except Exception as e:
    print(e)

# close connection
cursor.close()
connection.close()

In [37]:
# insert cleaned_owHistory.csv data into db

# connect to db
connection = psycopg2.connect("dbname=dbproject-sorare user=postgres password=Test1234") # use your user & password
connection.autocommit = True
cursor = connection.cursor()

# move data to public, because of access rights (ON CONFLICT clause is only available in PostgreSQL 9.5 and later)
sql = '''COPY OwnershipHistory(address,transferDate,priceEuro,transferType, blockchain, nftID)
FROM 'C:/Users/Public/cleaned_owHistory.csv'
DELIMITER ','
ENCODING 'UTF8'
CSV HEADER;'''

try:
    cursor.execute(sql)
except Exception as e:
    print(e)

# close connection
cursor.close()
connection.close()